In [86]:
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix, precision_score, mean_absolute_error
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn import svm
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.neural_network import MLPClassifier, MLPRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
import sklearn

np.set_printoptions(precision=4)

In [87]:
data = pd.read_csv('../Processed Data/RaceData/final_df.csv')

display(data)

,season,round,weather_warm,weather_cold,weather_dry,weather_wet,weather_cloudy,driver,grid,podium,...,constructor_minardi,constructor_prost,constructor_red_bull,constructor_renault,constructor_sauber,constructor_team_lotus,constructor_toro_rosso,constructor_toyota,constructor_tyrrell,constructor_williams
0,1983,1,False,False,True,False,False,keke_rosberg,1,15,...,0,0,0,0,0,0,0,0,0,1
1,1983,1,False,False,True,False,False,prost,2,6,...,0,0,0,1,0,0,0,0,0,0
2,1983,1,False,False,True,False,False,tambay,3,4,...,0,0,0,0,0,0,0,0,0,0
3,1983,1,False,False,True,False,False,piquet,4,1,...,0,0,0,0,0,0,0,0,0,0
4,1983,1,False,False,True,False,False,warwick,5,7,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15293,2022,22,True,False,False,False,False,kevin_magnussen,16,17,...,0,0,0,0,0,0,0,0,0,0
15294,2022,22,True,False,False,False,False,gasly,17,14,...,0,0,0,0,0,0,0,0,0,0
15295,2022,22,True,False,False,False,False,bottas,18,15,...,0,0,0,0,0,0,0,0,0,0
15296,2022,22,True,False,False,False,False,albon,19,13,...,0,0,0,0,0,0,0,0,0,1


In [88]:
df = data.copy()

train = df[df.season <2022]
X_train = train.drop(['driver', 'podium'], axis = 1)
y_train = train.podium

scaler = StandardScaler()
X_train = pd.DataFrame(scaler.fit_transform(X_train), columns = X_train.columns)

In [89]:
comparison_dict ={'model':[],
                  'params': [],
                  'score': []}

# Implement Model

In [90]:
model = sklearn.linear_model.LogisticRegression()
model.fit(X_train, y_train)

/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

# Evaluate Model

In [91]:
def score_classification(model):
    score = 0
    for circuit in df[df.season == 2022]['round'].unique():

        test = df[(df.season == 2022) & (df['round'] == circuit)]
        X_test = test.drop(['driver', 'podium'], axis = 1)
        y_test = test.podium

        #scaling
        X_test = pd.DataFrame(scaler.transform(X_test), columns = X_test.columns)

        # make predictions
        #test = np.transpose(np.array(model.predict(X_test, return_std=True)))
        test = model.predict(X_test)
        #prediction_df = pd.DataFrame(test, columns = ['y_mean', 'y_std'])
        prediction_df = pd.DataFrame(test, columns=['y_pred'])
        prediction_df['actual'] = y_test.reset_index(drop = True)

        #prediction_df = prediction_df[prediction_df.y_std < 2]

        #score += mean_absolute_error(prediction_df.actual, prediction_df.y_mean)
        score += mean_absolute_error(prediction_df.actual, prediction_df.y_pred)

    model_score = score / df[df.season == 2019]['round'].unique().max()
    return model_score

In [92]:
model_score = score_classification(model)

comparison_dict['model'].append('gpr')
comparison_dict['score'].append(model_score)

display(comparison_dict)

{'model': ['gpr'], 'params': [], 'score': [3.6190656379510866]}

In [93]:
# Run the test on Bahrain data
circuits = df[df.season == 2022]['round'].unique()
test_circuit = df[(df.season == 2022) & (df['round'] == circuits[0])]

X_test = test_circuit.drop(['driver', 'podium'], axis = 1)
y_test = test_circuit.podium

#scaling
X_test = pd.DataFrame(scaler.transform(X_test), columns = X_test.columns)

# make predictions
#test = np.transpose(np.array(model.predict(X_test, return_std=True)))
test = pd.DataFrame(model.predict_proba(X_test))

display(test_circuit['driver'])

driverId_dict = {
'hamilton' : 'Lewis Hamilton',
'alonso' : 'Fernando Alonso',
'vettel' : 'Sebastian Vettel',
'gasly' : 'Pierre Gasly',
'ricciardo' : 'Daniel Ricciardo',
'bottas' : 'Valtteri Bottas',
'kevin_magnussen' : 'Kevin Magnussen',
'max_verstappen' : 'Max Verstappen',
'sainz' : 'Carlos Sainz',
'ocon' : 'Esteban Ocon',
'stroll' : 'Lance Stroll',
'leclerc' : 'Charles Leclerc',
'norris' : 'Lando Norris',
'russell' : 'George Russell',
'latifi' : 'Nicholas Latifi',
'tsunoda' : 'Yuki Tsunoda',
'mick_schumacher' : 'Mick Schumacher',
'zhou' : 'Zhou Guanyu',
'albon' : 'Alexander Albon',
'perez' : 'Sergio Perez',
'hulkenberg' : 'Nico Hulkenberg'
}


14939            leclerc
14940     max_verstappen
14941              sainz
14942              perez
14943           hamilton
14944             bottas
14945    kevin_magnussen
14946             alonso
14947            russell
14948              gasly
14949               ocon
14950    mick_schumacher
14951             norris
14952              albon
14953               zhou
14954            tsunoda
14955         hulkenberg
14956          ricciardo
14957             stroll
14958             latifi
Name: driver, dtype: object

In [94]:
df_420 = test
df_420['DNF'] = test[20] + test[21] + test[22] + test[23] + test[24] + test[25] + test[26]
df_420.drop([20, 21, 22, 23, 24, 25, 26], axis = 1)

df_420['Driver'] = test_circuit['driver'].reset_index(drop=True)
df_420['Driver'] = df_420['Driver'].map(driverId_dict)

display(df_420)

df_420.to_csv('../Processed Data/Dummy Probability Outputs/test_bahrain.csv')

,0,1,2,3,4,5,6,7,8,9,...,19,20,21,22,23,24,25,26,DNF,Driver
0,0.188364,0.211727,0.170843,0.117339,0.062954,0.034080,0.026039,0.017170,0.014014,0.012008,...,0.009334,0.007555,0.004899,0.000826,0.000183,0.000013,1.325808e-05,1.103349e-05,0.013501,Charles Leclerc
1,0.375294,0.226111,0.108785,0.074610,0.051711,0.020231,0.011435,0.017675,0.010069,0.008201,...,0.008780,0.003203,0.003272,0.000065,0.000191,0.000004,1.551935e-06,2.059105e-06,0.006739,Max Verstappen
2,0.190362,0.242743,0.140602,0.088392,0.071459,0.048609,0.028948,0.021338,0.015079,0.012260,...,0.009033,0.009499,0.007535,0.000901,0.000209,0.000021,1.214172e-05,8.510055e-06,0.018186,Carlos Sainz
3,0.136012,0.109434,0.183746,0.102437,0.079767,0.062015,0.056996,0.046961,0.045094,0.017843,...,0.016657,0.007802,0.007561,0.000013,0.000466,0.000003,3.654014e-06,3.567251e-06,0.015852,Sergio Perez
4,0.142388,0.114253,0.077071,0.082570,0.101282,0.063216,0.091622,0.026853,0.057304,0.024685,...,0.007209,0.008938,0.014168,0.002719,0.001191,0.000039,1.723709e-05,1.729366e-06,0.027074,Lewis Hamilton
5,0.002102,0.111663,0.027056,0.115378,0.042328,0.024971,0.047949,0.053741,0.073328,0.059832,...,0.024858,0.008012,0.002637,0.001907,0.000337,0.000036,7.808085e-07,3.535150e-07,0.012930,Valtteri Bottas
6,0.000067,0.001267,0.000022,0.000470,0.044552,0.056480,0.084215,0.081279,0.054806,0.065751,...,0.047155,0.010240,0.012731,0.000003,0.000001,0.000001,1.150082e-06,4.804238e-06,0.022983,Kevin Magnussen
7,0.038842,0.069322,0.042488,0.088991,0.081941,0.096162,0.081188,0.066426,0.036607,0.032820,...,0.029610,0.024880,0.026400,0.004331,0.001377,0.000181,7.282316e-05,1.949478e-06,0.057243,Fernando Alonso
8,0.014541,0.053804,0.061811,0.064745,0.105898,0.052248,0.111270,0.034229,0.087167,0.040476,...,0.011681,0.014206,0.014182,0.003208,0.001856,0.000042,1.643602e-05,1.689123e-05,0.033527,George Russell
9,0.002934,0.021855,0.034370,0.055351,0.056803,0.070628,0.065641,0.058502,0.074715,0.041359,...,0.051080,0.022181,0.019059,0.003215,0.002213,0.000127,4.625094e-05,3.526120e-05,0.046877,Pierre Gasly
